In [1]:
import numpy as np
import csv
import gensim as gn
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string
from scipy.sparse import csc_matrix, save_npz, lil_matrix
import json
import os, re, math
import multiprocessing as mp
import pandas as pd

In [2]:
counter = 0
masterx2i, masteri2x = {}, {}

In [3]:
def createDicts(file_name):
    global counter
    print(file_name)
    wc = pd.read_csv("wc2/wc_"+file_name+'.csv')
    wc = wc.drop("Unnamed: 0", axis=1)
    wc.columns = ["word", "count"]
    # print(wc)
    for index, row in wc.iterrows():
        if row["word"] not in masterx2i.keys():
            masterx2i[row["word"]] = counter
            masteri2x[counter] = row["word"]
            counter = counter+1
            # print(counter)

In [4]:
def savei2x(i2x):
    w = csv.writer(open("i2xdf.csv", "w"))
    for key, val in i2x.items():
        w.writerow([key, val])
        
def savex2i(x2i):
    w = csv.writer(open("x2idf.csv", "w"))
    wc_files = [f for f in os.listdir("wc2") if os.path.isfile(os.path.join("wc2", f))]
    # print(wc_files)
    for i in range(len(wc_files)):
        file_name = wc_files[i][-14:-4]
        createDicts(file_name)
    for key, val in x2i.items():
        w.writerow([key, val])

In [5]:
savei2x(masteri2x)
savex2i(masterx2i)

2018-05-28
2018-03-05
2018-05-14
2018-06-04
2018-08-20
2018-05-21
2018-06-18
2018-03-19
2018-03-12
2018-04-30
2018-06-25
2018-04-02
2018-07-30
2018-08-06
2018-04-09
2018-07-02
2018-07-09
2018-06-11
2018-01-29
2018-07-16
2018-08-13
2018-04-23
2018-09-03
2018-01-22
2018-05-07
2018-08-27
2018-09-17
2018-09-10


In [6]:
i2xdf = pd.DataFrame(masteri2x.items(), columns=['Index', 'Word'])
x2idf = pd.DataFrame(masterx2i.items(), columns=['Word', 'Index'])

In [7]:
i2xdf.to_csv("i2xdf.csv")
x2idf.to_csv("x2idf.csv")

i2xdfread = pd.read_csv("i2xdf.csv")
x2idfread = pd.read_csv("x2idf.csv")

In [61]:
x2idfread

,Unnamed: 0,Word,Index
0,0,canada,0
1,1,collateral,1
2,2,damage,2
3,3,u,3
4,4,tariffs,4
5,5,knew,5
6,6,michael,6
7,7,cohen,7
8,8,unleashed,8
9,9,rage,9


In [101]:
def createNewMatrices(matrix):
    largePPMI = lil_matrix([len(x2idfread),len(x2idfread)])
    print(largePPMI)
    loadedwc = pd.read_csv("./wc2/wc_"+matrix+".csv", index_col=0)
    loadedwnd = pd.read_csv("./wnd2/wnd_"+matrix+".csv", index_col=0)
    wnd.columns=['e','w1','w2','count'] # e is for empty column
    wndSum = loadedwnd['2'].sum()
    wcSum = loadedwc['1'].sum()
    for index, row in loadedwnd.iterrows():
        
        if index % 10000 == 0:
            print(f'finished {index/len(loadedwnd):.2%} of the PPMI matrix: '+matrix)

        word1 = row['0']
        word2 = row['1']
        
        wc1row = loadedwc.loc[loadedwc['0'] == word1].values.tolist()
        wc2row = loadedwc.loc[loadedwc['0'] == word2].values.tolist()
        wc1 = wc1row[0][1]
        wc2 = wc2row[0][1]
        
        top = row['2'] * len(i2xdfread)
        bot = wc1 * wc2
        PPMIvalue = math.log(float(top/bot),2)
        print(PPMIvalue)
        
        if PPMIvalue < 0:
            PPMIvalue = 0
            
        index1 = i2xdfread.loc[i2xdfread['Word']==word1]['Index'].values
        print(index1[0])
        index2 = i2xdfread.loc[i2xdfread['Word']==word2]['Index'].values
        print(index2[0])
        
        largePPMI[index1][index2] = PPMIvalue
        largePPMI[index2][index1] = PPMIvalue
        # print(largePPMI[indexforword2][indexforword1])
    save_npz("/opt/data/Twitter-Embedding-Analysis/updatedmatrices/UpdatedPPMI_"+matrix+".npz", largePPMI)
    print(matrix+" finished creating")
    

In [102]:
oldwnd = [f for f in os.listdir("./wnd2") if os.path.isfile(os.path.join("./wnd2", f))]

In [103]:
for i in range(len(oldwnd)):
    matrix_name = oldwnd[i][-14:-4]
    createNewMatrices(matrix_name)

  (0, 0)	216189
  (0, 1)	216189


/home/zanepage/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


finished 0.00% of the PPMI matrix: 2018-06-04
5.621232149494528
370
6176


ValueError: row index 370 out of bounds